# `fuzzymatcher` examples

## Basic usage - `link_table`

In the most basic usage, the user provides `fuzzymatcher` with two pandas dataframes, indicating which columns to join on.

The central output of `fuzzymatcher` is the `link_table`.

For each record in the left table, the link table includes one or more possible matching records from the right table.

The user can then inspect the link table and decide which matches to retain, e.g. by choosing a score threshold ( `match_score > chosen_threshold` ) or just choosing the best match ( `match_rank == 1` )

In [1]:
import fuzzymatcher
import pandas as pd

df_left = pd.read_csv("tests/data/left_1.csv")
df_left

,id,fname,mname,lname,dob,another_field
0,1,Will,James,Johnston,20/05/1980,other data
1,2,James,Paul,Smith,15/06/1990,more data
2,3,Jody,Liz,Brown,20/05/1960,another thing
3,4,David,James,Williams,01/01/2000,final thing


In [2]:
df_right = pd.read_csv("tests/data/right_1.csv")
df_right

,id,name,middlename,surname,date,other
0,1,William,J,Johnston,20/05/1980,other data
1,2,James,Paul,Smith,15/06/1990,more data
2,3,Jodi,Elizabeth,Brown,20/05/1961,another thing


In [3]:
# Columns to match on from df_left
left_on = ["fname", "mname", "lname",  "dob"]

# Columns to match on from df_right
right_on = ["name", "middlename", "surname", "date"]

# Note that if left_id_col or right_id_col are admitted a unique id will be autogenerated
fuzzymatcher.link_table(df_left, df_right, left_on, right_on, left_id_col = "id", right_id_col = "id")

,id_left,id_right,match_score,match_rank,fname,name,mname,middlename,lname,surname,dob,date
0,1,1,0.286227,1,Will,William,James,J,Johnston,Johnston,20/05/1980,20/05/1980
1,2,2,0.561542,1,James,James,Paul,Paul,Smith,Smith,15/06/1990,15/06/1990
2,3,3,0.231445,1,Jody,Jodi,Liz,Elizabeth,Brown,Brown,20/05/1960,20/05/1961
3,4,2,0.053619,1,David,James,James,Paul,Williams,Smith,01/01/2000,15/06/1990


## Basic usage - `fuzzy_left_join`

A second option is to use `fuzzy_left_join`, which automatically links the two dataframes based on the highest-scoring match.

In [4]:
import fuzzymatcher
import pandas as pd

df_left = pd.read_csv("tests/data/left_1.csv")
df_right = pd.read_csv("tests/data/right_1.csv")
left_on = ["fname", "lname",  "dob"]
right_on = ["name", "surname", "date"]

fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on, right_on)

,best_match_score,id_left,fname,mname,lname,dob,another_field,id_right,name,middlename,surname,date,other
0,0.262926,1,Will,James,Johnston,20/05/1980,other data,1.0,William,J,Johnston,20/05/1980,other data
1,0.424492,2,James,Paul,Smith,15/06/1990,more data,2.0,James,Paul,Smith,15/06/1990,more data
2,0.211877,3,Jody,Liz,Brown,20/05/1960,another thing,3.0,Jodi,Elizabeth,Brown,20/05/1961,another thing
3,NaN,4,David,James,Williams,01/01/2000,final thing,NaN,NaN,NaN,NaN,NaN,NaN


# Basic example with real data
### Matching the names of Local Authorities provided by Office for National Statistics with the names provided by Ordnance Survey

We would usually join this data on the Local Authority District (LAD) Codes (e.g. E06000001 = Hartlepool), but sometimes these are unavailable.  In this example, we fuzzy match on the name, but provide the LAD code to demonstate it has worked.

In [5]:
ons = pd.read_csv("tests/data/las_ons.csv")
os = pd.read_csv("tests/data/las_os.csv")

df_joined = fuzzymatcher.fuzzy_left_join(ons, os, left_on = "lad16nm", right_on = "name")
rename = {"lad16cd": "ons_code", "code": "os_code", "lad16nm": "ons_name", "name": "os_name"}
df_joined = df_joined.rename(columns=rename)
col_order = ["best_match_score", "ons_name", "os_name", "ons_code", "os_code"]
df_joined[col_order].head()

,best_match_score,ons_name,os_name,ons_code,os_code
0,0.149865,Hartlepool,Hartlepool (B),E06000001,E06000001
1,0.149865,Middlesbrough,Middlesbrough (B),E06000002,E06000002
2,0.509971,Redcar and Cleveland,Redcar and Cleveland (B),E06000003,E06000003
3,0.552514,Stockton-on-Tees,Stockton-on-Tees (B),E06000004,E06000004
4,0.149865,Darlington,Darlington (B),E06000005,E06000005


We can get a sense of match quality by measuring how often the fuzzy matcher got it right:

In [6]:
num_records = len(df_joined)
correct_binary = (df_joined["ons_code"] == df_joined["os_code"])
perc_correct = correct_binary.sum()/num_records

"The percentage of codes which were correctly matched was {:,.1f}%".format(perc_correct*100)

'The percentage of codes which were correctly matched was 99.7%'

# Advanced usage - configuring the matcher

`fuzzymatcher` uses a number of components, each one of which can be re-written or adapted by the user:

* **`data_preprocessor`**:  Responsible for normalising strings, removing punctuation etc.
* **`datagetter`**:  Responsible for finding a list of possible matches for each df_left record in df_right
* **`scorer`**:  Responsible for computing a match score, given a record from df_left and df_right respectively

The main `link_table` and `fuzzy_left_join` convenience functions use these components under the hood.  See [here](https://github.com/RobinL/fuzzymatcher/blob/master/fuzzymatcher/__init__.py) for how this work.

This section provides a few examples of how an advanced user can compose these components to create a custom matcher

## Example 1:  Replacing the default sqlite datagetter with the cartesian datagetter

In [7]:
from fuzzymatcher.data_getter_cartesian import DataGetterCartesian
from fuzzymatcher.matcher import Matcher

dg = DataGetterCartesian()

m = Matcher(data_getter = dg)

df_left = pd.read_csv("tests/data/left_3.csv")
df_right = pd.read_csv("tests/data/right_3.csv")

on = ["first_name", "surname", "dob", "city"]

m.add_data(df_left, df_right, on, on)

m.match_all()
lt = m.get_formatted_link_table()
lt.head() # Note, because df_left and df_right are 100 records each, this table is 10,000 records long

,__id_left,__id_right,match_score,match_rank,first_name_left,first_name_right,surname_left,surname_right,dob_left,dob_right,city_left,city_right
0,0_left,0_right,0.173329,1,Noah,Noah,John,John,1979-05-29,1979-07-02,London,London
1,0_left,38_right,-0.015993,2,Noah,Samuel,John,Curtis,1979-05-29,1979-12-23,London,London
2,0_left,92_right,-0.015993,3,Noah,Bella,John,Jenkins,1979-05-29,1979-03-13,London,London
3,0_left,3_right,-0.021862,4,Noah,Jasmine,John,tewNo,1979-05-29,1973-07-29,London,London
4,0_left,14_right,-0.058994,5,Noah,Esme,John,Hunter,1979-05-29,1980-05-06,London,London


The Cartesian matcher considers more potential matches, but its performance is considerably worse

In [8]:
lt = fuzzymatcher.link_table(df_left, df_right, on, on)
lt.head()

,__id_left,__id_right,match_score,match_rank,first_name_left,first_name_right,surname_left,surname_right,dob_left,dob_right,city_left,city_right
0,0_left,0_right,0.356414,1,Noah,Noah,John,John,1979-05-29,1979-07-02,London,London
1,1_left,1_right,0.841329,1,William,William,Leo,Lowe,1996-01-29,1996-01-29,Ipswich,Ipswich
2,2_left,2_right,0.565822,1,Muhammad,Riley,lRey,Muhammad,1987-04-13,1987-04-13,Bradford,Bradford
3,3_left,3_right,0.055515,1,iasJne,Jasmine,Newton,tewNo,1973-09-15,1973-07-29,London,London
4,4_left,4_right,0.384755,1,Brown,Jacob,Jacob,Brown,1994-10-11,1994-07-26,London,London
